Implementé el método variational mixture of gaussians. No alcancé a hacer mucha vectorización. Calculé todos menos dos factores del lower bound, sin embargo uno de ellos me daba problemas por lo que no lo agregué al código de forma completa.

In [ ]:
%matplotlib inline

import numpy as np

import time
import math
from numpy.linalg import inv
import matplotlib
from scipy.stats import binom
#from scipy.stats import gamma
from scipy.stats import laplace
from scipy.optimize import minimize
from scipy import integrate
from scipy.optimize import minimize
from scipy.misc import derivative
from matplotlib import pyplot as plt
from scipy.stats import multivariate_normal
from scipy.special import gamma 
from scipy.special import psi
from scipy.stats import dirichlet
from scipy.stats import wishart
from scipy.misc import logsumexp

from matplotlib.patches import Ellipse


##Código de ploteo de elipses de https://github.com/joferkington/oost_paper_code/blob/master/error_ellipse.py
def plot_point_cov(points, nstd=2, ax=None, **kwargs):
    """
    Plots an `nstd` sigma ellipse based on the mean and covariance of a point
    "cloud" (points, an Nx2 array).
    Parameters
    ----------
        points : An Nx2 array of the data points.
        nstd : The radius of the ellipse in numbers of standard deviations.
            Defaults to 2 standard deviations.
        ax : The axis that the ellipse will be plotted on. Defaults to the 
            current axis.
        Additional keyword arguments are pass on to the ellipse patch.
    Returns
    -------
        A matplotlib ellipse artist
    """
    pos = points.mean(axis=0)
    cov = np.cov(points, rowvar=False)
    return plot_cov_ellipse(cov, pos, nstd, ax, **kwargs)

def plot_cov_ellipse(cov, pos, nstd=2, ax=None, **kwargs):
    """
    Plots an `nstd` sigma error ellipse based on the specified covariance
    matrix (`cov`). Additional keyword arguments are passed on to the 
    ellipse patch artist.
    Parameters
    ----------
        cov : The 2x2 covariance matrix to base the ellipse on
        pos : The location of the center of the ellipse. Expects a 2-element
            sequence of [x0, y0].
        nstd : The radius of the ellipse in numbers of standard deviations.
            Defaults to 2 standard deviations.
        ax : The axis that the ellipse will be plotted on. Defaults to the 
            current axis.
        Additional keyword arguments are pass on to the ellipse patch.
    Returns
    -------
        A matplotlib ellipse artist
    """
    def eigsorted(cov):
        vals, vecs = np.linalg.eigh(cov)
        order = vals.argsort()[::-1]
        return vals[order], vecs[:,order]

    if ax is None:
        ax = plt.gca()

    vals, vecs = eigsorted(cov)
    theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))

    # Width and height are "full" widths, not radius
    width, height = 2 * nstd * np.sqrt(vals)
    ellip = Ellipse(xy=pos, width=width, height=height, angle=theta, linewidth=3, fill=False, **kwargs)

    ax.add_artist(ellip)
    return ellip

In [ ]:
np.seterr(invalid='raise', divide='raise')
#Functions                                                                                                  
#(10.64)
def f_E_muk_lk(D, b_k, v_k, x_n, m_k, W_k): 
    #print("La 10.64")
    #print("la resta : "+ str(x_n - m_k))
    #print("la W : "+ str(W_k))
    #print("El producto :" + str(np.dot(np.transpose((x_n - m_k)), np.dot(W_k,(x_n - m_k)))))
    return D*(1/b_k) + v_k* np.dot(np.transpose((x_n - m_k)), np.dot(W_k,(x_n - m_k)))

#(10.65)
def f_E_ln_lk(D, v_k, W_k):
    A = 0
    for i in range(1,D+1):
        #print(i)
        #print("Division "+ str((v_k + 1 - i)/2))
        #print("Psi " + str(psi((v_k + 1 - i)/2)))
        A += psi((v_k + 1 - i)/2)
    #print("Valor A " + str(A))
    
    #print("Determinante " + str(np.linalg.det(W_k)))
    return A + D*np.log(2) + np.log(np.linalg.det(W_k))

#(10.66)
f_E_ln_pik = lambda a_k,alpha: psi(a_k) - psi(sum(alpha))

#(10.46)
def f_ln_pnk(a_k ,alpha ,D , b_k, v_k, x_n, m_k, W_k):
    #Retorna el valor de la función y el valor ln_pik para guardarlo en la matriz ln_pik
    ln_pik = f_E_ln_pik(a_k,alpha)    
    E_ln_l_k = f_E_ln_lk(D, v_k, W_k)
    return ln_pik + (1/2)*E_ln_l_k - (D/2)*np.log(2*np.pi) - (1/2)*f_E_muk_lk(D, b_k, v_k, x_n, m_k, W_k) , ln_pik, E_ln_l_k

#(10.49)
def f_r_nk(ln_pnk, ln_pnk_array):
    #print(ln_pnk)
    #print(ln_pnk_array)
    #print(logsumexp(ln_pnk_array))
    #print(ln_pnk -  logsumexp(ln_pnk_array))
    return np.exp(ln_pnk -  logsumexp(ln_pnk_array))

#(10.51)
def f_N_k(r_nk, k): 
    #print("F_N_k " + str(r_nk))
    #print A[[0, 1, 2], range(len(A))]
    #print("columna 1 " + str(r_nk[k,N]))
    #print("columna 2 " + str(r_nk[:,k]))
    #print("Suma " + str(sum(r_nk[:,k])))
    return sum(r_nk[:,k]) + 0.000000000000000000000000000001   


#(10.52)
def f_x_kmean(r_nk, x_n,N_k,N,D): 
    #print("f_x_mean")
    #print("El rnk " + str(r_nk))
    #print("El xn " + str(x_n))
    #print("El N_k " + str(N_k))
    result = 0
    for n in range(N):
        result += r_nk[n]*x_n[n]
    #print(N_k)
    #print(x_n)
    #np.tile(c,(4,1))
    #print(np.tile(r_nk,(D,1))
    #result = sum(np.tile(r_nk,(D,1))*x_n)
    return (1/N_k) * result #sum(np.multiply(r_nk,x_n))

#(10.53)
def f_S_k(r_nk, x_n, x_k, N_k, N, D):
    #print("En el f_S_k")
    #print("El r_nk "+ str(r_nk))
    #print("El x_n " + str(x_n))
    #print("El x_k "+ str(x_k))
    #print("El N_k "+ str(N_k))  
    result = np.matrix(np.zeros((D,D)))
    #print("Los result "+ str(result))
    for n in range(N):       
        x_n_matrix = np.matrix(x_n[n])        
        x_k_matrix = np.matrix(x_k)
        #print("La x_n matrix "+ str(x_n_matrix))
        #print("La x_k matrix "+ str(x_k_matrix))
        #print("El r_nk n " + str(r_nk[n]))
        #print("La resta de matrices "+ str((x_n_matrix-x_k_matrix) ))
        #print("La resta transpuesta "+ str(np.transpose(x_n_matrix-x_k_matrix)))
        #print("El producto de matrices " +str(( np.transpose(x_n_matrix-x_k_matrix) * (x_n_matrix-x_k_matrix) ) ))
        #print("El producto de matrices por r_nk " +str( r_nk[n]*( np.transpose(x_n_matrix-x_k_matrix) * (x_n_matrix-x_k_matrix) ) ))
        result = np.add(result, r_nk[n]*(  np.transpose(x_n_matrix-x_k_matrix) * (x_n_matrix-x_k_matrix)  ) )
        #print("El resultado parcial "+ str(result))
    
    #print("El result procesado "+ str((1/N_k) * result))
    return (1/N_k) * result

#(10.60)
def f_b_k(N_k,b_0): 
    return b_0 + N_k

#(10.61)
def f_m_k(b_k,b_0,m_0,N_k,x_kmean): 
    return (1/b_k)*(b_0*m_0 + N_k*x_kmean)

#(10.62)
def W_kinverse(N_k,S_k, W_0,m_0,b_0,x_kmean): 
    #print("10.62")
    #print("Nk*S_k "+ str(N_k*S_k))
    #print("x_kmean - m_0 "+ str(x_kmean - m_0))
    #print("Tranpsose * non " + str( ((b_0*N_k)/(b_0 + N_k))*  np.transpose(np.matrix(x_kmean - m_0)) * (np.matrix(x_kmean - m_0))))
    #print("W_0 inverte " + str(inv(W_0)))
    return inv(W_0) + N_k*S_k + ((b_0*N_k)/(b_0 + N_k))*  np.transpose(np.matrix(x_kmean - m_0)) * (np.matrix(x_kmean - m_0))

#(10.63)
v_k = lambda v_0, N_k: v_0 + N_k

#(10.69)
f_E_pik = lambda a_k, N_k, a_0, N, K: (a_k + N_k) / (K*a_0 + N)


La celda debajo no es para ser evaluada, la utilizé para testear algunas cosas, lo dejo para mi propio uso en instancias futuras.

In [ ]:
#ejemplo = np.array([[1.0,0.5],[0.2,0.0],[1.0,1.0]])
#print(ejemplo)
#N = len(ejemplo)
#print("El ejemplo")
#print(N)
#K = 2
#D = 2
#b = []
#v = []
#m = []
#W = []
#alpha = []
#for k in range(K):
#    b.append(0.5)
#    v.append(D+1)
#    m.append([0.0 + k/K ,0.5])
#    W.append([])
#    alpha.append(2.)
#    for i in range(D):
#        W[k].append([])
#        for j in range(D):
#            if(i == j):
#                W[k][i].append(1.)
#            else:
#                W[k][i].append(0.2)

#b = np.array(b)
#v = np.array(v)
#m = np.array(m)
#W = np.array(W)

#print(b[2])
#print(v[2])
#print(m[2])
#print(W[2])
#print(alpha[2])
#print(ejemplo[0])

#(10.64)
#ej1 = f_E_muk_lk(D, b[2], v[2], ejemplo[0], m[2], W[2])
#print(ej1)

#(10.65)
#ej2 = f_E_ln_lk(D, v[2], W[2])
#print(ej2)

#(10.66)
#ej3 = f_E_ln_pik(alpha[2], alpha)
#print(ej3)
#print(psi(alpha[2])- psi(K*alpha[2]))


#(10.46)
#ej4 = f_ln_pnk(alpha[2] ,alpha ,D , b[2], v[2], ejemplo[0], m[2], W[2])
#ej5 = f_ln_pnk(alpha[1] ,alpha ,D , b[1], v[1], ejemplo[0], m[1], W[1])
#print(ej4)
#print(ej3 + (1/2)*ej2 - (D/2)*np.log(2*np.pi) - (1/2)*ej1)
#print(ej5)
#(10.49)
#ej6 = f_r_nk(ej4, [ej4, ej5])
#print(ej6)

#r_nk_array = np.zeros((N, K))
#ln_pnk_array = np.zeros((N, K))
#N_k_array = 1.0*np.ones(K)
#x_k_mean_array = np.zeros((K,D))
#S_k_array = [0.0]*K


#(10.51)
#for n in range(0,N):
    #print(n)
#    for k in range(K):
#        ln_pnk = f_ln_pnk(alpha[k] ,alpha ,D , b[k], v[k], ejemplo[n], m[k], W[k])        
#        ln_pnk_array[n][k] = ln_pnk[0]
        #print("Los ln_pnk " + str(ln_pnk))
    #print("Arreglo ln_pnk " + str(ln_pnk_array))
    #Terminado un n, guardar r_nk (requiere todo los ln_pnk)
#    for k2 in range(K):
#        r_nk_array[n][k2] = f_r_nk(ln_pnk_array[n][k2], ln_pnk_array[n])

#print("Arreglo rnk "+ str(r_nk_array))
#print("Salimos del loop n")
#for k3 in range(K):
#    print("hola0")
#    N_k_array[k3] = f_N_k(r_nk_array, k3)
#    x_k_mean = f_x_kmean(r_nk_array[:,k3], ejemplo,N_k_array[k3],N,D)
#    print(x_k_mean)
#    x_k_mean_array[k3][0] = x_k_mean[0]
#    x_k_mean_array[k3][1] = x_k_mean[1]
#    S_k_array[k3] = f_S_k(r_nk_array[:,k3], ejemplo, x_k_mean, N_k_array[k3], N, D)
    


In [ ]:
#(10.58)
f_a_k =  lambda a_0, N_k:  a_0 + N_k
                               

#(10.45)
def f_ln_q_z(z,p,N,K,D,alpha,b,v,x,m,W):  
    result = 0
    for n in range(N):
        for k in range(K):
            result += z[n][k]*ln_pnk(a[k] ,alpha ,D , b[k], v[k], x[n], m[k], W[k]) 
    return result


    
#(10.47)
def f_q_z(z,p,N,K,D,alpha,b,v,x,m,W):
    result = 1
    for n in range(N):
        for k in range(K):
            result *= (f_r_nk( np.exp(f_ln_pnk (alpha[k], alpha, D, b[k], v[k], x[n], m[k], W[k]) ) ))**(z[n][k])  #Dudas con dimensiones
    return result



In [ ]:
#(B.23)
def f_C(alpha):
    alpha_gorro = sum(alpha)
    #print("Sin gama gorro "+ str(alpha_gorro))
    #print("El alpha gorro "+ str(gamma(alpha_gorro)))
    denominador = 1.0
    for i in alpha:
        #print("El elemento que se gammariza "+ str(i))
        denominador *= gamma(i) 
        #print("Denominadores intermedios "+ str(denominador))
    #print("Denominador final " + str(denominador))
    #print("Resultado sin log "+ str((gamma(alpha_gorro) / denominador)))
    result = 0
    if (math.isinf(gamma(alpha_gorro) )):
        result = 1000000000000000
    else:
        result = (gamma(alpha_gorro) / denominador)
    return result

#(10.73)
def f_E_ln_p_pi(a_0, ln_pik):     
    #print("El a_0" + str(a_0))
    #print("El ln_pik " + str(ln_pik))
    #print("La suma por el a_0 " + str((a_0[0] - 1)*sum(ln_pik)))
    #print("El log de C "+ str(np.log(f_C(a_0))))
    return np.log(f_C(a_0)) + (a_0[0] - 1)*sum(ln_pik)   ##El log puede tener un problema

#(10.72)
def f_ln_p_z(r_nk, ln_pik, N, K):
    #print("El r_nk " + str(r_nk))
    #print("El ln_pik " + str(ln_pik))
    ln_p_z = 0
    for n in range(N):
        for k in range(K):
            ln_p_z += r_nk[n][k]*ln_pik[k]
    #print("Valor ln_p_z "+ str(ln_p_z))
    return ln_p_z

#(10.75)
def f_E_ln_q_z(r_nk, N,K):
    E_ln_q_z = 0
    #print("El r_nk es " + str(r_nk))
    for n in range(N):
        for k in range(K):
            E_ln_q_z += r_nk[n][k] * np.log(r_nk[n][k])    
    #print("E_ln_q_z es " + str(E_ln_q_z))
    return E_ln_q_z
    
#(10.76)
def f_E_ln_q_pi(alpha, ln_pik, K): 
    E_ln_q_pi = 0.0
    #print("El alpha es " + str(alpha))
    #print("El ln_pik es " + str(ln_pik))
    for k in range(K):
        E_ln_q_pi += (alpha[k] - 1)*ln_pik[k]
    E_ln_q_pi += E_ln_q_pi +  np.log(f_C(alpha))
    #print("E_ln_q_pi es " + str(E_ln_q_pi + np.log(f_C(alpha))))  
    return E_ln_q_pi 
         
#(10.71)
def f_E_ln_p_x(N_k, ln_l_k, b, v, S_k, W_k, x_mean, m, K ):
    #print("El N_k " + str(N_k))
    #print("El ln_l_k "+ str(ln_l_k))
    #print("El b "+ str(b))
    #print("El v "+ str(v))
    #print("El S_k "+ str(S_k))
    #print("El W_k "+ str(W_k))
    #print("El x_mean_k "+ str(x_mean))
    #print("El m "+ str(m))
    E_ln_p_x = 0
    
    for k in range(K):
        #print("El trace " + str(np.trace(S_k[k]*W_k[k])))
        E_ln_p_x += N_k[k]* ( ln_l_k[k] - D*(1/b[k]) -v[k]*np.trace(S_k[k]*W_k[k]) - v[k]*( np.dot(np.transpose((x_mean[k] - m[k])), np.dot(W_k[k],(x_mean[k] - m[k])))) -D*np.log(2*np.pi))
    
    E_ln_p_x = (E_ln_p_x)/2
    return E_ln_p_x

#(10.57)
q_pi = lambda pi, alpha: dirichlet(pi, alpha)

#Priors
#(10.39)
p_pi = lambda pi, a_0: dirichlet(pi,a_0)

#(10.40)
def p_mu_l(mu,m_0,b_0, l, W_0, v_0, K):
    result = 1
    for k in range(K):
        result *= gaussian(mu[k], m_0, 1/(b_0*l[k])) *  wishart.pdf(l[k], v_0, W_0)[0]  #Revisar esto
    return result

#(10.59)
q_muk_lk = lambda mu_k,m_k,b_k, l_k, W_k: gaussian(mu_k,m_k, 1/(b_k*l_k)) * wishart(l_k, v_k, W_k)
    

In [ ]:
#Lectura datos
#datos_clustering.txt
file_name = 'datos_clustering.txt'
def read_file(file_name):
    file = open(file_name, 'r')
    X = []
    for line in file:
        line = line.strip()
        line = line.split(',')
        for i in range(len(line)):
            line[i] = float(line[i])
        X.append(line)
    return np.array(X)

X = read_file(file_name)

In [ ]:
def normalize(x,D):
    x_min = []
    x_max = []
    a = 0.0
    b = 0.0
    
    for i in range(D):    
        x_min.append(min(x[:,i]))
        x_max.append(max(x[:,i]))

    for i in range(len(x)):
        for d in range(D):
            x[i][d] = (x[i][d] - x_min[d]) / (x_max[d] - x_min[d])
            
    return x

In [ ]:
##Test normalize
#print(X)
#X2 = normalize(X, len(X[0]))
#print(X2)

In [ ]:
#Vector inicialization
def vector_initialize(K,D):
    W_0 = []
    a_0 = []
    b_0 = []
    alpha = []
    mu = []
    b = []
    m = []
    m_0 = []
    W = []
    v = []
    v_0 = []
    pi = []
    ln_pi = []
    ln_l_k = []
    for i in range(K):
        
        a_0.append(0.0001)
        b_0.append(1.)
        alpha.append(0.0001)
        mu.append([0.5,0.5]) #Hacerlo general para las dimensiones
        b.append(1.)
        m.append([0.0 + i/K,0.0+i/K])
        m_0.append([0.0 + i/K,0.0+i/K])
        v.append(D+1)
        v_0.append(D+1)
        pi.append(1/K)
        ln_pi.append(0.0)
        ln_l_k.append(0.0)
        W.append([])
        W_0.append([])
        for j in range(D):
            W[i].append([])
            W_0[i].append([])
            for k in range(D):
                if(j == k):
                    W[i][j].append(2.)
                    W_0[i][j].append(2.)
                else:
                    W[i][j].append(0.)
                    W_0[i][j].append(0.)
                
    return np.array(W_0), np.array(W), np.array(a_0), np.array(alpha), np.array(b_0), np.array(b), np.array(m), np.array(m_0) , np.array(v), np.array(v_0), np.array(pi),np.array(mu), D,np.array(ln_pi),np.array(ln_l_k)

In [ ]:
#Final plot
%matplotlib qt
#Algoritmo
def variational_mixture(file_name,K, iterations):
    x = read_file(file_name)
    D = len(x[0])
    x = normalize(x, D)
    W_0, W, a_0, alpha, b_0, b, m, m_0 , v,v_0, pi, mu,D, ln_pik, ln_l_k = vector_initialize(K,D)
    N = len(x)
    x = np.array(x)
    r_nk_array = np.zeros((N, K))
    ln_pnk_array = np.zeros((N, K))
    N_k_array = 10*np.ones(K)
    x_k_mean_array = np.zeros((K,D))
    S_k_array = [0]*K

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.scatter(x[:,0], x[:,1])
    ax.scatter(m[:,0], m[:,1], color = (0,1,0))
    plt.pause(1)

    for i in range(iterations):
        print("*********Iteracion "+ str(i) + "*********************")

        for n in range(N):
            #print(n)
            for k in range(K):
                ln_pnk = f_ln_pnk(alpha[k] ,alpha ,D , b[k], v[k], x[n], m[k], W[k])
                ln_pnk_array[n][k] = ln_pnk[0]
                #Valor de (10.66) para el lower bound
                ln_pik[k] = ln_pnk[1]
                ln_l_k[k] = ln_pnk[2]

            #Terminado un n, guardar r_nk (requiere todo los ln_pnk)
            for k2 in range(K):
                r_nk_array[n][k2] = f_r_nk(ln_pnk_array[n][k2], ln_pnk_array[n])

        for k3 in range(K):
            N_k_array[k3] = f_N_k(r_nk_array,k3)
            x_k_mean = f_x_kmean(r_nk_array[:,k3], x,N_k_array[k3],N, D)
            x_k_mean_array[k3][0] = x_k_mean[0]
            x_k_mean_array[k3][1] = x_k_mean[1]
            S_k_array[k3] = f_S_k(r_nk_array[:,k3], x, x_k_mean, N_k_array[k3], N, D)

        #Update parameters
        for k4 in range(K):
            #(10.58)
            alpha[k4] = a_0[k4] + N_k_array[k4]
            #(10.60)
            b[k4] = f_b_k(N_k_array[k4],b_0[k4])
            #(10.61)
            m[k4] = f_m_k(b[k4],b_0[k4],m_0[k4], N_k_array[k4], x_k_mean_array[k4])
            #(10.62)
            W[k4] =   inv( W_kinverse(N_k_array[k4],S_k_array[k4], W_0[k4],m_0[k4],b_0[k4],x_k_mean_array[k4]) ) 
            #(10.63)
            v[k4] = v_0[k4] + N_k_array[k4]
            #(10.69)
            pi[k4] = f_E_pik(alpha[k4], N_k_array[k4], a_0[k4], N, K)

        #Calculate lower bound
        #(10.73) y B.23
        E_ln_p_pi = f_E_ln_p_pi(a_0, ln_pik)
        #(10.72)
        ln_p_z = f_ln_p_z(r_nk_array, ln_pik, N, K)
        #(10.75)
        #E_ln_q_z = f_E_ln_q_z(r_nk_array, N,K)
        #(10.76)
        #E_ln_q_pi = f_E_ln_q_pi(alpha, ln_pik, K)                #Manda problemas 
        #(10.71)
        #E_ln_p_x = f_E_ln_p_x(N_k_array, ln_l_k, b, v, S_k_array, W, x_k_mean_array, m, K )


        #Visualización    
        plt.clf()
        ax = fig.add_subplot(1, 1, 1)
        ax.scatter(x[:,0], x[:,1])
        W_plot = []
        #Set covariance matrix
        for k in range(len(W)):
            if v[k] == D + 1:
                W_plot.append(inv(W[k]) * 0)
            else:
                if pi[k] > 0.001:
                    W_plot.append(inv(W[k]) * (1/(v[k]  - D - 1)))
                    ax.scatter(m[k,0], m[k,1], color = (0,1,0))          
                    plot_cov_ellipse(W_plot[k], m[k], nstd=2, ax=ax)
        plt.pause(0.05)    
        plt.show()


In [ ]:
file_name = 'datos_clustering.txt'
iterations = 70
K = 5
variational_mixture(file_name, K, iterations)